In [12]:
from sage.all import * 

def unitary_divisors(n):
    return [x for x in divisors(n) if gcd(x,n/x)==1]

def kk(a,b):
    return gcd(a,b)**2/(a*b)

def plus(a,b):
    return a*b/(gcd(a,b)**2)

def mult(a,b):
    return gcd(a,b)

def compositionToDivisor(c):
    s = Composition(c).to_subset()
    if len(s)==0:
        return 1
    return prod([nth_prime(x) for x in s])

def rad(n):
    return prod(prime_divisors(n))

def divisorToComposition(d,n):
    if rad(d)==d:
        s = set([prime_pi(x) for x in prime_divisors(d)])
        return Composition(from_subset=(s,n))

def compositionKernel(c1,c2):
    a = compositionToDivisor(c1)
    b = compositionToDivisor(c2)
    return kk(a,b)

def primorial(n):
    return prod(primes(nth_prime(n)))


def pivoted_cholesky(M,max_rank=int(30)):
    import tensorflow_probability as tfp
    import tensorflow as tf
    import numpy as np
    piv_chol = (tfp.math.pivoted_cholesky(M,max_rank = max_rank, diag_rtol=float(0.001), name=None)).numpy()
    return piv_chol


# https://mathoverflow.net/questions/373475/a-geometric-approach-to-the-odd-perfect-number-problem
# embedding:
def jordanTotient(n,k=2):
    return n**2*prod([1-1/p**2 for p in prime_divisors(n)])

# embedding:
# https://mathoverflow.net/questions/373475/a-geometric-approach-to-the-odd-perfect-number-problem
def phi(C,axis=0):
    from scipy.sparse import dok_matrix,lil_matrix,csr_matrix,csc_matrix
    import numpy as np
    n = int(np.sum(list(C)))
    print("n = ",n)
    A = compositionToDivisor(C)
    print("A =",A)
    print("P_n = ",primorial(n-1))
    if axis == 1:
        S = csr_matrix((int(1),int(2**(n-1))), dtype=np.float64)
        for d in divisors(A):
            ds = divisorToComposition(d,n).to_subset()
            x = int(np.sum([2**(i-1) for i in ds]))            
            S[0,x]         = float(1.0/A)*np.sqrt(jordanTotient(d,k=2))
    elif axis == 0:
        S = dok_matrix((int(2**(n-1)),int(1)), dtype=np.float64)
        for d in divisors(A):
            ds = divisorToComposition(d,n).to_subset()
            x = int(np.sum([2**(i-1) for i in ds]))            
            S[x,0]         = float(1.0/A)*np.sqrt(jordanTotient(d,k=2))
    return S    

# embedding of all Compositions(n) in a dok_matrix
def phiN(n,maxN=8):
    from scipy.sparse import dok_matrix,lil_matrix,csr_matrix,csc_matrix
    import numpy as np
    if n > maxN:
        return None
    CC = Compositions(n)
    lC = 2**(n-1)
    S = dok_matrix((lC,lC),dtype=np.float64)
    #print(S.shape)
    c = 0
    #print(CC)
    for C in CC:
        #print(C)
        A = compositionToDivisor(list(C))
        for d in divisors(A):
            ds = divisorToComposition(d,n).to_subset()
            x = int(np.sum([2**(i-1) for i in ds]))
            S[c,x] = float(1.0/A)*np.sqrt(jordanTotient(d,k=2))
        c+=1    
    return S

def fit_neighbors(n):
    X = phiN(n,maxN=8)
    CC = list(Compositions(n))
    from sklearn.neighbors import NearestNeighbors as NN
    neigh = NN()
    neigh = neigh.fit(X)
    return neigh,CC

def get_neighbors(neigh,CC,C,n_neighbors=5,reverse=True):
    v = phi(C,axis=1)
    dists, inds = (neigh.kneighbors(v, n_neighbors, return_distance=True))
    ll = []
    for i in inds[0]:
        ll.append(CC[i])
    if reverse:
        return ll[-1::-1]
    else:    
        return ll    

neigh,CC = fit_neighbors(8)
ll = get_neighbors(neigh,CC,[2,1,1,2,1,1],n_neighbors=5)
print(ll)

n =  8
A = 23205
P_n =  30030
[[1, 2, 1, 2, 1, 1], [2, 2, 2, 1, 1], [3, 1, 2, 1, 1], [1, 1, 1, 1, 2, 1, 1], [2, 1, 1, 2, 1, 1]]


In [16]:
import music21 as m21
score = m21.stream.Score()
tempo  = 150
tm = m21.tempo.MetronomeMark(number=tempo)
score.append(tm)

def getPadicDigits(q=2/3,p=2,prec=100):
    p = p
    RDigits = Qp(p=p,prec=prec,type="lattice-cap",print_mode="bars",print_sep=",")
    R = RDigits
    digits = ([ int(x) for x in str(R(q)).replace(".","").split(",") if len(x)>0])
    #digits.reverse()
    return digits[0:prec]


vollist = [128//8*k for k in [3,4,5,6]]

def harmonic(t,y0,y1,periodendauer):
    T = periodendauer
    #X = matrix([[np.sin(np.pi/2+phi0),1],[np.sin(np.pi/2),1]])
    #print(X)
    A = (y0-y1)/2
    B = (y0+y1)/2
    #print(A,B)
    frequenz = 1.0/T
    return int(np.round(A*np.cos(2*np.pi*frequenz*t)+B,0))

def getHarmonicFromList(t,ll,T):
    periodendauer = T
    y0 = 0
    y1 = len(ll)-1
    return ll[harmonic(t,y0,y1,periodendauer)]

def convertPitchNrToNote(pitchNr):
    Z12ToPitch = dict(zip(range(12),["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]))
    octave = pitchNr // 12
    pitch = pitchNr % 12
    
    note = Z12ToPitch[pitch]    
    
    return note +  str(octave)

parts = []
scale = [60 + x for x in [0,2,4,5,7,9,11]]

drum = list(range(41,48+1))

class WarmPad(m21.instrument.Instrument):
    '''
    >>> p = instrument.Piano()
    >>> p.instrumentName
    'Piano'
    >>> p.midiProgram
    0
    '''

    def __init__(self):
        super().__init__()

        self.instrumentName = 'Warm Pad'
        self.instrumentAbbreviation = 'W.P.'
        self.midiProgram = 89

        self.lowestNote = m21.pitch.Pitch('A0')
        self.highestNote = m21.pitch.Pitch('C8')

        
def minkowskiSum(A,B):
    return ([a+b for a in A for b in B])

def k_Minkowski(k,A):
    if k < 0:
        return None
    
    if k == 1:
        return A
    else:
        return minkowskiSum(k_Minkowski(k-1,A),A)


def cyclicExtension(nn):
    k = len(nn)
    ll = [n for n in nn]
    for i in range(k):
        ll.extend([nn[(j+i)%k] for j in range(k)])
    return ll    

def applyPermutationToList(perm,ll):
    if len(perm)==len(ll) and sorted(list(set(perm)))==list(range(len(ll))):
        return [ll[perm[i]] for i in range(len(perm))]

def getBarNumber(duration):
    import numpy as np
    return int(np.floor(duration/4.0))

def getSmallestPeriodicRational(periodlength=1,prime=7):
    n = 2
    while True:
        Zn = Zmod(n)
        for x in range(1,n):
            if gcd(x,prime)==1 and gcd(n,prime)==1 and multiplicative_order(Zn(prime))==periodlength:
                return(-x/n)
        n+=1    

def cumSum(ll):
    return [sum(ll[0:i]) for i in range(len(ll)+1)]
        
piano = m21.instrument.Piano()
woodblock = m21.instrument.Woodblock()
tomtom = m21.instrument.TomTom()
percussion = tomtom
warmpad = WarmPad()
marimba = m21.instrument.Marimba()
harp = m21.instrument.Harp()
string = m21.instrument.StringInstrument()

nrInst = 8
for k in range(nrInst):
    lh = m21.stream.Part()
    #lh.append(m21.instrument.Piano())
    #lh.append(WarmPad())
    if k%10 in [0]:
        lh.append(piano)
    elif k%10 in [1]:
        inst = piano #m21.instrument.AcousticBass()
        #inst.midiChannel = 9
        #inst.midiProgram = 53
        #inst.instrumentName = 'Orchestra Kit'
        #inst.instrumentAbbreviation = 'O.K.'
        print(inst)
   # 1 Standard Kit
   # 9 Room Kit
   # 17 Power Kit
   # 25 Electronic Kit
   # 26 TR-808 Kit
   # 33 Jazz Kit
   # 41 Brush Kit
   # 49 Orchestra Kit
   # 57 Sound FX Kit
   # 128 Percussion      
        lh.append(piano)
    elif k%10 in [2,3]:
        lh.append(piano) #m21.instrument.Piano())
    elif k%10 in [4,5]:
        lh.append(harp) #m21.instrument.Harp())
    elif k%10 in [6,7]:
        lh.append(warmpad) #m21.instrument.Marimba())
    elif k%10 in [8,9]:
        lh.append(piano) #WarmPad())
    parts.append(lh)


#shuffle(parts)    
    
prec = 256*20

import numpy as np

Npartition = 5
op = OrderedPartitions(Npartition).list()[1:]

shuffle(op)
changeAfterNSeconds = 5
last_tim = -changeAfterNSeconds
cnt = 0
nBars = 2**7
print(nBars)
if prec < nBars:
    print("......")
    
N = 2*3
possibleParts = [2,4,8]
#print(numbers)
cnt = 0
numbers = [getSmallestPeriodicRational(periodlength=d,prime=7) for d in divisors(N) if d > 1]
restsNumbers = [getSmallestPeriodicRational(periodlength=d,prime=2) for d in divisors(N) if d > 1]
print("restsNumbers = ",restsNumbers)


lB = floor(log(nBars)/log(2))+1

print(nrInst)
onOffInsts = [range(x*nrInst//lB,(x+1)*nrInst//lB) for x in range(lB)]
instsToPlay = []
for b in range(2**lB):
    digits2 = Integer(b).digits(2,padto=lB)
    print(digits2)
    playInst = []
    for x in range(nrInst):
        if any([x in onOffInsts[i] for i in range(lB) if digits2[i]==1]):
            pI = 1
        else:
            pI = 0
        playInst.append(pI)    
    print(playInst) 
    instsToPlay.append(playInst)


cnt = 0
x = [2,1,1,2,1,1]
neigh,CC = fit_neighbors(8)
LL = []
for i in range(len(parts)):
    digits = getPadicDigits(q=numbers[i%len(numbers)],p=7,prec=prec)
    rests = getPadicDigits(q=restsNumbers[i%len(restsNumbers)],p=2,prec=prec)
    durs = getPadicDigits(q=numbers[1],p=2,prec=prec)
    nPartitions = getPadicDigits(q=numbers[i%len(numbers)],p=3,prec=prec)
    m = len(digits)
    #print(i,numbers[i])
    #print(digits)
    print(i,nPartitions)
    #print(durs)
    sumDur = 0
    sumNN = 0
    #cnt = 0
    for bn in range(1,2**lB+1):
        time_in_minutes = bn*1/tempo*1.0
        tim = int(np.round(time_in_minutes*60/changeAfterNSeconds,0)*changeAfterNSeconds)
        
        playInst = instsToPlay[bn-1]
        volume = getHarmonicFromList(sum(playInst),vollist,T=2*len(parts))
        print("volume",volume,tim,time_in_minutes,bn,cnt)
        if tim-last_tim==changeAfterNSeconds and tim in [changeAfterNSeconds*x for x in cumSum([1,2,4,8,16,8,4])]:
            if len(LL)==0:
                ll = get_neighbors(neigh,CC,x,n_neighbors=5,reverse=True)
                LL.extend(ll)
            #op = OrderedPartitions(nPart).list()[:0:-1]
            #shuffle(op)
            xdurs = LL.pop(0)
            #shuffle(xdurs)
            print(xdurs,x,LL)
            x = xdurs
            cnt += 1
            nPart = np.sum(list(xdurs))
        last_tim = tim
        j = 0
        for digit in digits[sumNN:(sumNN+len(xdurs))]:
            #isRest = (rests[j]==1)
            pitchNr = scale[digit]
            D = float(xdurs[j%len(xdurs)]*4.0/nPart)   
            j+=1
            sumDur += D#print(bn,bnDigits)                 
            
            if playInst[i]==0:
                n0 = m21.note.Rest()
            else:    
                if i%10 in [-1]:# violine
                    n0 = m21.note.Note(convertPitchNrToNote(pitchNr-24))
                else:
                    n0 = m21.note.Note(convertPitchNrToNote(pitchNr-(i%2)*24))
                #n0 = m21.note.Note(convertPitchNrToNote(pitchNr-24))    
                n0.volume.velocity = volume        
            duration = m21.duration.Duration(D)
            n0.duration = duration #getDurationInQuarterByEmphasis(emph,factor=factor)
            parts[i].append(n0)
        sumNN+=len(xdurs)

for part in parts:
    score.append(part)
            
score.insert(0,m21.metadata.Metadata())    
title = "Spiral Sea"
score.metadata.title = title

score.metadata.composer = 'musescore1983'
score.write("midi", fp=("_".join(title.split(" ")))+".mid")     
score.write("musicxml", fp=("_".join(title.split(" ")))+".musicxml")     
    

Piano
128
restsNumbers =  [-0.3333333333333333, -0.14285714285714285, -0.1111111111111111]
8
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 0, 0]
[0, 1, 1, 1, 0, 0, 0, 0]
[0, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 1, 0, 0, 0]
[0, 1, 

0 [0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0

volume 48 0 0.013333333333333334 2 5
volume 48 0 0.02 3 5
volume 48 0 0.02666666666666667 4 5
volume 48 0 0.03333333333333333 5 5
volume 48 0 0.04 6 5
volume 48 5 0.04666666666666667 7 5
n =  8
A = 23205
P_n =  30030
[1, 2, 1, 2, 1, 1] [2, 1, 1, 2, 1, 1] [[2, 2, 2, 1, 1], [3, 1, 2, 1, 1], [1, 1, 1, 1, 2, 1, 1], [2, 1, 1, 2, 1, 1]]
volume 64 5 0.05333333333333334 8 6
volume 48 5 0.06 9 6
volume 48 5 0.06666666666666667 10 6
volume 48 5 0.07333333333333333 11 6
volume 64 5 0.08 12 6
volume 48 5 0.08666666666666667 13 6
volume 64 5 0.09333333333333334 14 6
volume 64 5 0.1 15 6
volume 80 5 0.10666666666666667 16 6
volume 48 5 0.11333333333333333 17 6
volume 48 5 0.12 18 6
volume 48 10 0.12666666666666668 19 6
volume 64 10 0.13333333333333333 20 6
volume 48 10 0.14 21 6
volume 64 10 0.14666666666666667 22 6
volume 64 10 0.15333333333333332 23 6
volume 80 10 0.16 24 6
volume 48 10 0.16666666666666666 25 6
volume 64 10 0.17333333333333334 26 6
volume 64 10 0.18 27 6
volume 80 10 0.18666666666

volume 48 0 0.013333333333333334 2 9
volume 48 0 0.02 3 9
volume 48 0 0.02666666666666667 4 9
volume 48 0 0.03333333333333333 5 9
volume 48 0 0.04 6 9
volume 48 5 0.04666666666666667 7 9
[2, 1, 1, 2, 1, 1] [1, 1, 1, 1, 2, 1, 1] []
volume 64 5 0.05333333333333334 8 10
volume 48 5 0.06 9 10
volume 48 5 0.06666666666666667 10 10
volume 48 5 0.07333333333333333 11 10
volume 64 5 0.08 12 10
volume 48 5 0.08666666666666667 13 10
volume 64 5 0.09333333333333334 14 10
volume 64 5 0.1 15 10
volume 80 5 0.10666666666666667 16 10
volume 48 5 0.11333333333333333 17 10
volume 48 5 0.12 18 10
volume 48 10 0.12666666666666668 19 10
volume 64 10 0.13333333333333333 20 10
volume 48 10 0.14 21 10
volume 64 10 0.14666666666666667 22 10
volume 64 10 0.15333333333333332 23 10
volume 80 10 0.16 24 10
volume 48 10 0.16666666666666666 25 10
volume 64 10 0.17333333333333334 26 10
volume 64 10 0.18 27 10
volume 80 10 0.18666666666666668 28 10
volume 64 10 0.19333333333333333 29 10
volume 80 10 0.2 30 10
volume 

volume 48 5 0.06666666666666667 10 14
volume 48 5 0.07333333333333333 11 14
volume 64 5 0.08 12 14
volume 48 5 0.08666666666666667 13 14
volume 64 5 0.09333333333333334 14 14
volume 64 5 0.1 15 14
volume 80 5 0.10666666666666667 16 14
volume 48 5 0.11333333333333333 17 14
volume 48 5 0.12 18 14
volume 48 10 0.12666666666666668 19 14
volume 64 10 0.13333333333333333 20 14
volume 48 10 0.14 21 14
volume 64 10 0.14666666666666667 22 14
volume 64 10 0.15333333333333332 23 14
volume 80 10 0.16 24 14
volume 48 10 0.16666666666666666 25 14
volume 64 10 0.17333333333333334 26 14
volume 64 10 0.18 27 14
volume 80 10 0.18666666666666668 28 14
volume 64 10 0.19333333333333333 29 14
volume 80 10 0.2 30 14
volume 80 10 0.20666666666666667 31 14
volume 80 15 0.21333333333333335 32 14
[2, 1, 1, 2, 1, 1] [1, 1, 1, 1, 2, 1, 1] []
volume 48 15 0.22 33 15
volume 48 15 0.22666666666666666 34 15
volume 48 15 0.23333333333333334 35 15
volume 64 15 0.24 36 15
volume 48 15 0.24666666666666667 37 15
volume 64 

volume 48 5 0.06666666666666667 10 18
volume 48 5 0.07333333333333333 11 18
volume 64 5 0.08 12 18
volume 48 5 0.08666666666666667 13 18
volume 64 5 0.09333333333333334 14 18
volume 64 5 0.1 15 18
volume 80 5 0.10666666666666667 16 18
volume 48 5 0.11333333333333333 17 18
volume 48 5 0.12 18 18
volume 48 10 0.12666666666666668 19 18
volume 64 10 0.13333333333333333 20 18
volume 48 10 0.14 21 18
volume 64 10 0.14666666666666667 22 18
volume 64 10 0.15333333333333332 23 18
volume 80 10 0.16 24 18
volume 48 10 0.16666666666666666 25 18
volume 64 10 0.17333333333333334 26 18
volume 64 10 0.18 27 18
volume 80 10 0.18666666666666668 28 18
volume 64 10 0.19333333333333333 29 18
volume 80 10 0.2 30 18
volume 80 10 0.20666666666666667 31 18
volume 80 15 0.21333333333333335 32 18
[1, 1, 1, 1, 2, 1, 1] [3, 1, 2, 1, 1] [[2, 1, 1, 2, 1, 1]]
volume 48 15 0.22 33 19
volume 48 15 0.22666666666666666 34 19
volume 48 15 0.23333333333333334 35 19
volume 64 15 0.24 36 19
volume 48 15 0.24666666666666667 3

5 [0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0

volume 48 0 0.02666666666666667 4 25
volume 48 0 0.03333333333333333 5 25
volume 48 0 0.04 6 25
volume 48 5 0.04666666666666667 7 25
n =  8
A = 23205
P_n =  30030
[1, 2, 1, 2, 1, 1] [2, 1, 1, 2, 1, 1] [[2, 2, 2, 1, 1], [3, 1, 2, 1, 1], [1, 1, 1, 1, 2, 1, 1], [2, 1, 1, 2, 1, 1]]
volume 64 5 0.05333333333333334 8 26
volume 48 5 0.06 9 26
volume 48 5 0.06666666666666667 10 26
volume 48 5 0.07333333333333333 11 26
volume 64 5 0.08 12 26
volume 48 5 0.08666666666666667 13 26
volume 64 5 0.09333333333333334 14 26
volume 64 5 0.1 15 26
volume 80 5 0.10666666666666667 16 26
volume 48 5 0.11333333333333333 17 26
volume 48 5 0.12 18 26
volume 48 10 0.12666666666666668 19 26
volume 64 10 0.13333333333333333 20 26
volume 48 10 0.14 21 26
volume 64 10 0.14666666666666667 22 26
volume 64 10 0.15333333333333332 23 26
volume 80 10 0.16 24 26
volume 48 10 0.16666666666666666 25 26
volume 64 10 0.17333333333333334 26 26
volume 64 10 0.18 27 26
volume 80 10 0.18666666666666668 28 26
volume 64 10 0.193333

volume 80 60 1.0066666666666666 151 32
volume 80 60 1.0133333333333334 152 32
volume 64 60 1.02 153 32
volume 80 60 1.0266666666666666 154 32
volume 80 60 1.0333333333333334 155 32
volume 80 60 1.04 156 32
volume 80 65 1.0466666666666666 157 32
volume 80 65 1.0533333333333332 158 32
volume 80 65 1.06 159 32
volume 96 65 1.0666666666666667 160 32
volume 48 65 1.0733333333333333 161 32
volume 64 65 1.08 162 32
volume 64 65 1.0866666666666667 163 32
volume 80 65 1.0933333333333333 164 32
volume 64 65 1.1 165 32
volume 80 65 1.1066666666666667 166 32
volume 80 65 1.1133333333333333 167 32
volume 80 65 1.12 168 32
volume 64 70 1.1266666666666667 169 32
volume 80 70 1.1333333333333333 170 32
volume 80 70 1.14 171 32
volume 80 70 1.1466666666666667 172 32
volume 80 70 1.1533333333333333 173 32
volume 80 70 1.16 174 32
volume 80 70 1.1666666666666667 175 32
volume 96 70 1.1733333333333333 176 32
volume 64 70 1.18 177 32
volume 80 70 1.1866666666666668 178 32
volume 80 70 1.1933333333333334 179

'/home/orges/notebooks/algorithmic_python_music/p-adic_numbers_and_compositions.xml'

In [25]:
for x in numbers:
    #print(x)
    print("".join([str(xx) for xx in getPadicDigits(q=x,p=7,prec=57)]))

515151515151515151515151515151515151515151515151515151515
053053053053053053053053053053053053053053053053053053053
346012346012346012346012346012346012346012346012346012346
